# <center>[Imperial College Loan Default Dataset](https://www.kaggle.com/c/loan-default-prediction)</center><br><br>
### <center>by Hector Cadeaux</center><br><br>
This competition asks you to determine whether a loan will default, as well as the loss incurred if it does default. Unlike traditional finance-based approaches to this problem, where one distinguishes between good or bad counterparties in a binary way, we seek to anticipate and incorporate both the default and the severity of the losses that result. In doing so, we are building a bridge between traditional banking, where we are looking at reducing the consumption of economic capital, to an asset-management perspective, where we optimize on the risk to the financial investor.

This competition is sponsored by researchers at Imperial College London.<br><br>
## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
loan = pd.read_csv('train_v2.csv',low_memory=False)


In [ ]:
sns.heatmap(loan.isna())
plt.show()

In [ ]:
loan.describe()

In [ ]:
loan_profile= loan.describe().T

In [ ]:
loan_profile['count'].value_counts()

In [ ]:
loan.loss.value_counts()

In [ ]:
loan

In [ ]:
loan_t = pd.read_csv('test_v2.csv',low_memory=False)

In [ ]:
loan.info()

In [ ]:
loan.dtypes

## Cleaning Train

In [ ]:
obj_col= [col for col in loan.columns if loan[col].dtype == 'object']
obj_col

In [ ]:
def large_floats(s):
    try:
        return float(s)
    except ValueError:
        if (type(s) != 'float64' or type(s) != 'int64') and (len(str(s)) > 6):
            s= str(s)
            to_the = str(len(s) -1)
            eight = str(round(float(s[:8])/10**7,7))
            return eight+'e'+to_the
    

In [ ]:
for col in loan.columns:
    nan_col_tr = []
    nnas = loan[col].isna().sum()
    if nnas > 0:
        print("{} has {} missing values".format(col,nnas))
        nan_col_tr.append(col)

In [ ]:
loan[obj_col]

In [ ]:
loan['f5'].value_counts()

In [ ]:
loan['f770'].value_counts()

In [ ]:
loan['f2'].value_counts()

In [ ]:
loan['f776'].value_counts().plot.barh()

In [ ]:
loan['f9'].value_counts().head(40).plot.barh()

In [ ]:
loan['f137'].value_counts().head(40).plot.bar()

In [ ]:
loan['f7'].value_counts()

In [ ]:
for col in loan.columns:
    loan[col] = loan[col].fillna(loan[col].median())

In [ ]:
sns.heatmap(loan.isna())
plt.show()

### Cleaning Test

In [ ]:
loan_t

In [ ]:
obj_col_t= [col for col in loan_t.columns if loan[col].dtype == 'object']
obj_col_t

In [ ]:
for clm in obj_col:
    loan[clm] = loan[clm].fillna(0)


In [ ]:
for clm in obj_col:
    loan[clm] = loan[clm].apply(large_floats)
    loan[clm] = loan[clm].astype('float64')
    print("{} is now {}".format(clm,loan[clm].dtype))

In [ ]:
sns.heatmap(loan[obj_col].isna())
plt.show()

In [ ]:
loan[obj_col].describe().T

In [ ]:
loan.describe().T

In [ ]:
for col in loan_t.columns:
    nan_col_t = []
    nnas = loan_t[col].isna().sum()
    if nnas > 0:
        print("{} has {} missing values".format(col,nnas))
        nan_col_t.append(col)

In [ ]:
for clm in obj_col_t:
    loan_t[clm] = loan_t[clm].fillna(0)

In [ ]:
for clm in obj_col_t:
    loan_t[clm] = loan_t[clm].apply(large_floats)
    loan_t[clm] = loan_t[clm].astype('float64')
    print("{} is now {}".format(clm,loan_t[clm].dtype))

In [ ]:
for col in loan_t.columns:
    loan_t[col] = loan_t[col].fillna(loan_t[col].median(skipna=True))

In [ ]:
sns.heatmap(loan_t.isna())

In [ ]:
loan_t[obj_col_t].describe().T

In [ ]:
from sklearn.feature_selection import f_classif, SelectKBest
y= loan['loss']
X = loan.drop(['id','loss'],axis=1)

In [ ]:
X.describe().T

In [ ]:
selector = SelectKBest(score_func=f_classif,k=10)
Xdisc= selector.fit_transform(X,y)
print(selector.get_support())

In [ ]:
df1 = pd.DataFrame(loan.drop(columns=['id','loss']).columns, columns=['feature'])

df2 = pd.DataFrame(selector.scores_, columns=['importance'])

importance = df1.join(df2)

importance.sort_values('importance', ascending=False).head(20)

In [ ]:
Xdisc.shape

In [ ]:
importance.sort_values('importance', ascending=False)[20:40]

In [ ]:
importance.describe()

In [ ]:
importance.importance.sum()

In [ ]:
importance.query('importance>=4.068193').importance.sum()

In [ ]:
importance.query('importance>=	1.918834').importance.sum()/importance.importance.sum()

In [ ]:
X.isna().any().sum()

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
trainX, validX, trainy, validy = train_test_split(X,y,train_size=.3,random_state=15)

In [ ]:
from sklearn.metrics import mean_squared_error
hgbr= HistGradientBoostingRegressor()
hgbr.fit(trainX, trainy)
y_pred=hgbr.predict(trainX)
mse= mean_squared_error(y_pred,trainy)

In [ ]:
mse

In [ ]:
valy_prd=hgbr.predict(validX)
msev= mean_squared_error(valy_prd,validy)


In [ ]:
msev

In [ ]:
trainy.describe()

In [ ]:
np.log1p(trainy).to_frame().query('loss >0').plot.hist(bins=100)

In [ ]:
trainy